In [29]:
# Import packages.
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras.layers import LSTM, Dense
import matplotlib
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from math import sqrt
from numpy import concatenate
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Conv3D, MaxPooling3D, UpSampling3D, Conv3DTranspose
from tensorflow.keras.models import Model

In [2]:
# Read dataset.
df = pd.read_csv("df_subset.csv")
# Drop unneeded columns caused by merging.
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
# Add calculated fields.
df["wind_speed"] = np.sqrt(df["u"]**2 + df["v"]**2)
df["wind_dir"] = np.sqrt(df["u"]**2 + df["v"]**2)

In [4]:
# Subset to a specific point.
df_subset = df[df["time"] == "2018-01-01 00:00:00"]
df_subset = df_subset.drop(columns="time")

In [5]:
df_subset

,lat,lon,level,z,pv,r,q,t,u,vo,v,u10,v10,t2m,tisr,tcc,tp,wind_speed,wind_dir
0,-87.1875,0.000,50,202029.2200,-5.748637e-05,0.165604,0.000003,235.22633,-1.544806,-0.000020,3.101584,-2.658455,-7.615494,250.72818,1753739.2,0.000000,0.000000,3.465003,3.465003
1,-87.1875,0.000,100,155859.2800,-1.886603e-05,0.588379,0.000003,228.20036,1.537735,0.000018,-1.535164,-2.658455,-7.615494,250.72818,1753739.2,0.000000,0.000000,2.172869,2.172869
2,-87.1875,0.000,150,129443.1700,-9.821752e-06,1.256958,0.000003,225.83513,0.664698,0.000031,-0.642512,-2.658455,-7.615494,250.72818,1753739.2,0.000000,0.000000,0.924470,0.924470
3,-87.1875,0.000,200,110969.9400,-8.707291e-06,7.723129,0.000006,220.10094,2.942966,0.000037,-5.742935,-2.658455,-7.615494,250.72818,1753739.2,0.000000,0.000000,6.453088,6.453088
4,-87.1875,0.000,250,97149.5000,-3.231180e-07,103.247470,0.000039,215.74178,5.463413,0.000117,-17.590302,-2.658455,-7.615494,250.72818,1753739.2,0.000000,0.000000,18.419218,18.419218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19798592,87.1875,354.375,600,36814.3480,6.627473e-07,74.370895,0.000176,238.48500,-3.123463,0.000022,0.158658,2.929847,-0.250536,253.69980,0.0,0.930111,0.000008,3.127490,3.127490
19798593,87.1875,354.375,700,26103.4220,8.170609e-07,68.261970,0.000282,245.26566,-3.168304,0.000012,0.568709,2.929847,-0.250536,253.69980,0.0,0.930111,0.000008,3.218941,3.218941
19798594,87.1875,354.375,850,12286.2660,1.374292e-06,100.999220,0.000490,248.69313,-1.254796,0.000025,0.628646,2.929847,-0.250536,253.69980,0.0,0.930111,0.000008,1.403463,1.403463
19798595,87.1875,354.375,925,6231.4062,1.022813e-06,100.366700,0.000526,250.29095,0.820169,0.000017,-0.245078,2.929847,-0.250536,253.69980,0.0,0.930111,0.000008,0.856003,0.856003


In [6]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
df_subset[['z', 'pv', 'r', 'q', 't', 'u', 'vo', 'v', 
           'u10', 'v10', 't2m', 'tisr', 'tcc', 'tp', 
           'wind_speed', 'wind_dir']] = mms.fit_transform(df_subset[['z', 'pv', 'r', 'q', 
                                                                     't', 'u', 'vo', 'v', 
                                                                     'u10', 'v10', 't2m', 
                                                                     'tisr', 'tcc', 'tp', 
                                                                     'wind_speed', 'wind_dir']])

In [7]:
df_subset

,lat,lon,level,z,pv,r,q,t,u,vo,v,u10,v10,t2m,tisr,tcc,tp,wind_speed,wind_dir
0,-87.1875,0.000,50,0.992335,0.023567,0.000373,0.000126,0.396853,0.285132,0.435631,0.521156,0.363119,0.277738,0.296161,0.347508,0.000000,0.000000,0.036502,0.036502
1,-87.1875,0.000,100,0.770115,0.299327,0.003378,0.000084,0.341033,0.308856,0.478336,0.479215,0.363119,0.277738,0.296161,0.347508,0.000000,0.000000,0.022632,0.022632
2,-87.1875,0.000,150,0.642971,0.363906,0.008130,0.000105,0.322242,0.302137,0.493499,0.487289,0.363119,0.277738,0.296161,0.347508,0.000000,0.000000,0.009232,0.009232
3,-87.1875,0.000,200,0.554058,0.371864,0.054094,0.000253,0.276685,0.319672,0.499843,0.441154,0.363119,0.277738,0.296161,0.347508,0.000000,0.000000,0.068575,0.068575
4,-87.1875,0.000,250,0.487539,0.431729,0.733115,0.001790,0.242052,0.339070,0.590358,0.333989,0.363119,0.277738,0.296161,0.347508,0.000000,0.000000,0.197015,0.197015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19798592,87.1875,354.375,600,0.197139,0.438769,0.527850,0.008150,0.422742,0.272982,0.483441,0.494536,0.514291,0.488456,0.330948,0.000000,0.930111,0.000943,0.032879,0.032879
19798593,87.1875,354.375,700,0.145587,0.439871,0.484426,0.013057,0.476613,0.272637,0.471295,0.498245,0.514291,0.488456,0.330948,0.000000,0.930111,0.000943,0.033860,0.033860
19798594,87.1875,354.375,850,0.079083,0.443849,0.717134,0.022682,0.503844,0.287364,0.485917,0.498788,0.514291,0.488456,0.330948,0.000000,0.930111,0.000943,0.014374,0.014374
19798595,87.1875,354.375,925,0.049941,0.441340,0.712637,0.024387,0.516538,0.303334,0.476866,0.490884,0.514291,0.488456,0.330948,0.000000,0.930111,0.000943,0.008497,0.008497


In [8]:
df["lat"].unique()

array([-87.1875    , -81.5625    , -75.9375    , -70.3125    ,
       -64.6875    , -59.0625    , -53.4375    , -47.8125    ,
       -42.1875    , -36.5625    , -30.9375    , -25.3125    ,
       -19.6875    , -14.0625    ,  -8.4375    ,  -2.8125    ,
         2.8125    ,   8.4375    ,  14.0625    ,  19.6875    ,
        25.3125    ,  30.9375    ,  36.5625    ,  42.1875    ,
        47.8125    ,  53.4375    ,  59.0625    ,  64.6875    ,
        70.3125    ,  75.9375    ,  81.5625    ,  87.1875    ,
        47.81250381])

In [9]:
df["lon"].unique()

array([  0.   ,   5.625,  11.25 ,  16.875,  22.5  ,  28.125,  33.75 ,
        39.375,  45.   ,  50.625,  56.25 ,  61.875,  67.5  ,  73.125,
        78.75 ,  84.375,  90.   ,  95.625, 101.25 , 106.875, 112.5  ,
       118.125, 123.75 , 129.375, 135.   , 140.625, 146.25 , 151.875,
       157.5  , 163.125, 168.75 , 174.375, 180.   , 185.625, 191.25 ,
       196.875, 202.5  , 208.125, 213.75 , 219.375, 225.   , 230.625,
       236.25 , 241.875, 247.5  , 253.125, 258.75 , 264.375, 270.   ,
       275.625, 281.25 , 286.875, 292.5  , 298.125, 303.75 , 309.375,
       315.   , 320.625, 326.25 , 331.875, 337.5  , 343.125, 348.75 ,
       354.375])

In [39]:
df[df["time"] <= '2018-01-02']["time"].unique()

array(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
       '2018-01-01 02:00:00', '2018-01-01 03:00:00',
       '2018-01-01 04:00:00', '2018-01-01 05:00:00',
       '2018-01-01 06:00:00', '2018-01-01 07:00:00',
       '2018-01-01 08:00:00', '2018-01-01 09:00:00',
       '2018-01-01 10:00:00', '2018-01-01 11:00:00',
       '2018-01-01 12:00:00', '2018-01-01 13:00:00',
       '2018-01-01 14:00:00', '2018-01-01 15:00:00',
       '2018-01-01 16:00:00', '2018-01-01 17:00:00',
       '2018-01-01 18:00:00', '2018-01-01 19:00:00',
       '2018-01-01 20:00:00', '2018-01-01 21:00:00',
       '2018-01-01 22:00:00', '2018-01-01 23:00:00'], dtype=object)

In [10]:
np.where(df["lat"].unique() == -14.0625)[0][0]

13

In [21]:
df["level"].unique()[1:]

array([ 100,  150,  200,  250,  300,  400,  500,  600,  700,  850,  925,
       1000], dtype=int64)

In [40]:
A_total = []
for N in (df[df["time"] <= '2018-01-02']["time"].unique()):
    A = []
    for i in  (df_subset["lat"].unique()):
        matrix = []
        for j in  (df_subset["lon"].unique()):
            row = []
            for k in  (df_subset["level"].unique()[1:]):
                row = row + [df_subset[(df_subset['lat'] == -87.1875) & 
                                       (df_subset['lon'] == 0) & 
                                       (df_subset['level'] == 1000)].drop(columns=["lat","lon","level"]).values[0]]
            matrix = matrix + [row]
        A = A + [matrix]
    A_total = A_total + [A]

In [41]:
A_array = np.array(A_total)

In [42]:
A_array.shape

(24, 32, 64, 12, 16)

In [43]:
input_layer = Input(shape=(32, 64, 12, 16), name="INPUT")
#x = data_augmentation(input_layer)
# Encoding
x = Conv3D(32, (2, 2, 2), activation='relu', padding='same')(input_layer)
x = MaxPooling3D((2, 2, 2))(x)
x = Conv3D(16, (2, 2, 2), activation='relu', padding='same')(x)
x = MaxPooling3D((2, 2, 2))(x)
x = Conv3D(16, (2, 2, 1), activation='relu', padding='same')(x)
x = MaxPooling3D((2, 2, 1))(x)

# Dense latent factors.
x = tf.keras.layers.Flatten()(x)
latent = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.Dense(4*8*3*16,activation='relu')(latent)
x = tf.keras.layers.Reshape((4,8,3,16))(x)

# Decoding
x = Conv3DTranspose(16, (2, 2, 1), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 1))(x)
x = Conv3DTranspose(16, (2, 2, 2), activation='relu', padding='same')(x)
x = UpSampling3D((2, 2, 2))(x)
x = Conv3DTranspose(32, (2, 2, 2), activation='relu', padding='same')(x)
x = UpSampling3D((2,2, 2))(x)
output_layer = Conv3D(16, (2, 2, 2), padding='same', name="OUTPUT")(x)


autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 INPUT (InputLayer)          [(None, 32, 64, 12, 16)]  0         
                                                                 
 conv3d_11 (Conv3D)          (None, 32, 64, 12, 32)    4128      
                                                                 
 max_pooling3d_11 (MaxPoolin  (None, 16, 32, 6, 32)    0         
 g3D)                                                            
                                                                 
 conv3d_12 (Conv3D)          (None, 16, 32, 6, 16)     4112      
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 8, 16, 3, 16)     0         
 g3D)                                                            
                                                                 
 conv3d_13 (Conv3D)          (None, 8, 16, 3, 16)      1040

In [44]:
A_array.shape

(24, 32, 64, 12, 16)

In [ ]:
autoencoder.fit(A_array, A_array, epochs=5, steps_per_epoch=10, batch_size=16)